In [41]:
import numpy as np
import matplotlib.pyplot as plt
from src.core.pybullet_core import PybulletCore
from src.utils import *

# Open Pybullet GUI

In [42]:
pb = PybulletCore()
pb.connect(robot_name = "indy7_v2", joint_limit=True, constraint_visualization = False)

******** ROBOT INFO ********
Robot name: indy7_v2
Robot type: indy7_v2
DOF: 6
Joint limit: True
Constraint visualization: False


Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\Users\rnbmarch\AppData\Local\anaconda3\envs\robotics\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\rnbmarch\AppData\Local\anaconda3\envs\robotics\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\rnbmarch\AppData\Local\anaconda3\envs\robotics\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\rnbmarch\OneDrive - postech.ac.kr\바탕 화면\robotics\mech439_pybullet_framework-main\src\core\pybullet_core.py", line 123, in _thread_main
    self.my_robot.robot_update()
  File "c:\Users\rnbmarch\OneDrive - postech.ac.kr\바탕 화면\robotics\mech439_pybullet_framework-main\src\core\pybullet_robot.py", line 57, in robot_update
    self._get_robot_states()      # update robot's states
  File "c:\Users\rnbmarch\OneDrive - postech.ac.kr\바탕 화면\robotics\mech439_pybullet_framework-main\src\c

# Move robot

In [43]:
pb.MoveRobot([0, 30, -120, 0, -90, 0], degree=True, verbose=True)
from time import sleep
sleep(3)

***** Set desired joint angle *****
[ 0.      0.5236 -2.0944  0.     -1.5708  0.    ]


# Get PyBullet robot's properties (Current states)

In [44]:
q = pb.my_robot.q
PRINT_YELLOW("Joint position (rad)", q.T)

qdot = pb.my_robot.qdot
PRINT_YELLOW("Joint velocity (rad/s)", qdot.T) #dq.T

p = pb.my_robot.p
PRINT_YELLOW("End-effector's pose in xyz-xi vector", p.T)

T_end = pb.my_robot.T_end
PRINT_YELLOW("End-effector's pose in SE3\n", T_end)

Jr = pb.my_robot.Jr
PRINT_YELLOW("Robot's jacobian\n", Jr)

g = pb.my_robot.g
PRINT_YELLOW("Gravity compensate term", g.T)

Joint position (rad): [[ 0.      0.5236 -2.0944  0.     -1.5708  0.    ]]
Joint velocity (rad/s): [[-0. -0.  0.  0.  0. -0.]]
End-effector's pose in xyz-xi vector: [[ 0.125  -0.2025  0.5772  0.     -3.1416  0.    ]]
End-effector's pose in SE3
: [[-1.     -0.     -0.      0.125 ]
 [-0.      1.     -0.     -0.2025]
 [ 0.     -0.     -1.      0.5772]
 [ 0.      0.      0.      1.    ]]
Robot's jacobian
: [[ 0.2025 -0.2777  0.112  -0.      0.112  -0.    ]
 [ 0.125   0.      0.      0.112  -0.     -0.    ]
 [ 0.      0.125   0.35   -0.199  -0.      0.    ]
 [ 0.     -0.     -0.     -1.     -0.     -0.    ]
 [-0.     -1.     -1.     -0.     -1.      0.    ]
 [-1.      0.      0.     -0.      0.      1.    ]]
Gravity compensate term: [[ 0.     -2.4299 14.7955 -3.342  -0.      0.    ]]


# Rotation & Transformation Matrix Utils

In [45]:
# Euler angle -> Rotation matrix
R = eul2Rot([90, 0, 0], seq='XYZ', degree=True)
print(R)

# Rotation matrix -> Euler angle
eul = Rot2eul(R, seq='XYZ', degree=True)
print(eul)

[[ 1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  1.  0.]]
[90.  0.  0.]


In [46]:
# xyz position + Euler angle -> Transformation matrix
T = xyzeul2SE3([0.2, 0.3, 0.4], [90, 0, 0], seq='XYZ', degree=True)
print(T)

# Transformation matrix -> xyz position + Euler angle
xyz = T[0:3, 3]
eul = Rot2eul(T[0:3, 0:3], seq='XYZ', degree=True)
print(xyz, eul)

[[ 1.   0.   0.   0.2]
 [ 0.   0.  -1.   0.3]
 [ 0.   1.   0.   0.4]
 [ 0.   0.   0.   1. ]]
[0.2 0.3 0.4] [90.  0.  0.]


# Matrix Exponential/Logarithm (Out of this course!)

In [47]:
R = eul2Rot([90, 0, 0], seq='XYZ', degree=True)
print(R)

# Rotation matrix -> 3-by-1 vector (log)
xi = Rot2Vec(R)
print(xi)

# 3-by-1 vector -> rotation matrix (exp)
R = Vec2Rot(xi)
print(R)

[[ 1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  1.  0.]]
[[1.5708]
 [0.    ]
 [0.    ]]
[[ 1.  0.  0.]
 [ 0. -0. -1.]
 [ 0.  1. -0.]]


In [48]:
T = xyzeul2SE3([0.2, 0.3, 0.4], [90, 0, 0], seq='XYZ', degree=True)
print(T)

# Transformation matrix -> 6-by-1 vector (log)
lamb = SE32Vec(T)
print(lamb)

# 6-by-1 vector -> transformation matrix (exp)
T = Vec2SE3(lamb)
print(T)

# Transformation matrix -> xyz-xi vector (decoupled: position (xyz) and orientation(xi))
xyz_xi = SE32PoseVec(T)
print(xyz_xi)

# xyz-xi vector -> transformation matrix
T = PoseVec2SE3(xyz_xi)
print(T)

[[ 1.   0.   0.   0.2]
 [ 0.   0.  -1.   0.3]
 [ 0.   1.   0.   0.4]
 [ 0.   0.   0.   1. ]]
[[1.5708]
 [0.    ]
 [0.    ]
 [0.2   ]
 [0.5498]
 [0.0785]]
[[ 1.   0.   0.   0.2]
 [ 0.  -0.  -1.   0.3]
 [ 0.   1.  -0.   0.4]
 [ 0.   0.   0.   1. ]]
[[0.2   ]
 [0.3   ]
 [0.4   ]
 [1.5708]
 [0.    ]
 [0.    ]]
[[ 1.   0.   0.   0.2]
 [ 0.  -0.  -1.   0.3]
 [ 0.   1.  -0.   0.4]
 [ 0.   0.   0.   1. ]]


# Inverse Kinematics

In [ ]:
T_goal = xyzeul2SE3([0, -0.5, 0.5], [0,45,0], seq='ZYZ', degree=True)

pb.add_debug_frames([T_goal])
print(T_goal)

q_i = pb.my_robot.q
qlist = np.zeros([6, 0])
qlist = np.concatenate((qlist, q_i), axis=1)
for _ in range(100):
    T_i = pb.my_robot.pinModel.FK(q_i)
    Jb_i = pb.my_robot.pinModel.Jb(q_i)

    R_i = T_i[0:3, 0:3]
    A_upper = np.concatenate((np.zeros([3, 3]), R_i), axis=1)
    A_lower = np.concatenate((np.eye(3), np.zeros([3, 3])), axis=1)
    A = np.concatenate((A_upper, A_lower), axis=0)

    Jv_i = A @ Jb_i
    
    R_goal = T_goal[0:3, 0:3]
    euler_goal = Rot2eul(R_goal, seq='ZYZ', degree=True) * np.pi / 180
    euler_i = Rot2eul(R_i, seq='ZYZ', degree=True) * np.pi / 180

    phi_i, theta_i, psi_i = euler_i
    Tr_i = np.array([[0, -np.sin(phi_i), np.cos(phi_i)*np.sin(theta_i)],
                     [0, np.cos(phi_i), np.sin(phi_i)*np.sin(theta_i)],
                     [1, 0, np.cos(theta_i)]])
    Tr_i = np.linalg.pinv(Tr_i)
    
    
    B_upper = np.concatenate((np.eye(3), np.zeros([3, 3])), axis=1)
    B_lower = np.concatenate((np.zeros([3, 3]), Tr_i), axis=1)
    B = np.concatenate((B_upper, B_lower), axis=0)

    Jr_i = B @ Jv_i

    #TODO: Define error
    p_err = 
 
    #TODO: Update q
    q_i = 

    qlist = np.concatenate((qlist, q_i), axis=1)
    pb.MoveRobot(q_i, degree=False)
    sleep(0.1)

import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(qlist.T*180/np.pi)
plt.legend(["q1", "q2", "q3", "q4", "q5", "q6"])
plt.xlabel("Step")
plt.ylabel("Joint angle (deg)")
plt.yticks([-180, -120, -60, 0, 60, 120, 180])
plt.show()